In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# dataset = pd.read_csv('/content/drive/MyDrive/Python-Dataset/News_Data.csv', sep=",", encoding='utf-8')
dataset = pd.read_csv('/content/drive/MyDrive/Python-Dataset/NewsCategorizer.csv', sep=",", encoding='utf-8')

# dataset = pd.read_csv('/content/NewsCategorizer.csv', sep=",", encoding='utf-8')

dataset['Category'] = dataset['category']
dataset.drop('category',axis='columns', inplace=True)

dataset['Text'] = dataset['headline'] + dataset['short_description']
dataset.drop('headline',axis='columns', inplace=True)
dataset.drop('short_description',axis='columns', inplace=True)

dataset.drop('links',axis='columns', inplace=True)
dataset.drop('keywords',axis='columns', inplace=True)

dataset.head()

Mounted at /content/drive


,Category,Text
0,WELLNESS,143 Miles in 35 Days: Lessons LearnedResting i...
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?Th...
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...
3,WELLNESS,"Oh, What a Difference She MadeIf you want to b..."
4,WELLNESS,"Green SuperfoodsFirst, the bad news: Soda brea..."


In [3]:
print("shape :- ", dataset.shape,"\n\n")
dataset.info()

shape :-  (50000, 2) 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  50000 non-null  object
 1   Text      50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
dataset['Category'].value_counts()

WELLNESS          5000
POLITICS          5000
ENTERTAINMENT     5000
TRAVEL            5000
STYLE & BEAUTY    5000
PARENTING         5000
FOOD & DRINK      5000
WORLD NEWS        5000
BUSINESS          5000
SPORTS            5000
Name: Category, dtype: int64

In [5]:
target_category = dataset['Category'].unique()
print(target_category)

['WELLNESS' 'POLITICS' 'ENTERTAINMENT' 'TRAVEL' 'STYLE & BEAUTY'
 'PARENTING' 'FOOD & DRINK' 'WORLD NEWS' 'BUSINESS' 'SPORTS']


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(dataset['Category'])
print(le.classes_)
dataset['CategoryId']=le.transform(dataset['Category'])

dataset

['BUSINESS' 'ENTERTAINMENT' 'FOOD & DRINK' 'PARENTING' 'POLITICS' 'SPORTS'
 'STYLE & BEAUTY' 'TRAVEL' 'WELLNESS' 'WORLD NEWS']


,Category,Text,CategoryId
0,WELLNESS,143 Miles in 35 Days: Lessons LearnedResting i...,8
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?Th...,8
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,8
3,WELLNESS,"Oh, What a Difference She MadeIf you want to b...",8
4,WELLNESS,"Green SuperfoodsFirst, the bad news: Soda brea...",8
...,...,...,...
49995,SPORTS,This Baseball Team Learned There's A Wrong Way...,5
49996,SPORTS,Some Young Spurs Fan Dabbed 38 Times In A Sing...,5
49997,SPORTS,Rasheed Wallace Ejected From Knicks-Suns Game ...,5
49998,SPORTS,Why Jake Plummer And Other NFL Players Are Pus...,5


In [7]:
# dataset['CategoryId'] = dataset['Category'].factorize()[0]

# dataset

In [8]:
text = dataset['Text']
text.head(10)

0    143 Miles in 35 Days: Lessons LearnedResting i...
1    Talking to Yourself: Crazy or Crazy Helpful?Th...
2    Crenezumab: Trial Will Gauge Whether Alzheimer...
3    Oh, What a Difference She MadeIf you want to b...
4    Green SuperfoodsFirst, the bad news: Soda brea...
5    Bad Love Advice: 9 Lessons To UnlearnBy Carey ...
6    The Happiest (And Unhappiest) States In The U....
7    Seaweed: The Green Superfood You're Not Eating...
8    Addicted to Food?If you look at our culture's ...
9    High Tech Works When It Enables High TouchFran...
Name: Text, dtype: object

In [9]:
category = dataset['Category']
category.head(10)

0    WELLNESS
1    WELLNESS
2    WELLNESS
3    WELLNESS
4    WELLNESS
5    WELLNESS
6    WELLNESS
7    WELLNESS
8    WELLNESS
9    WELLNESS
Name: Category, dtype: object

In [10]:
# Create a new pandas dataframe "category", which only has unique Categories, also sorting this list in order of CategoryId values
category = dataset[['Category', 'CategoryId']].drop_duplicates().sort_values('CategoryId')
category

,Category,CategoryId
40000,BUSINESS,0
10000,ENTERTAINMENT,1
30000,FOOD & DRINK,2
25000,PARENTING,3
5000,POLITICS,4
45000,SPORTS,5
20000,STYLE & BEAUTY,6
15000,TRAVEL,7
0,WELLNESS,8
35000,WORLD NEWS,9


In [11]:
def remove_tags(text):
  remove = re.compile(r'')
  return re.sub(remove, '', text)
  
dataset['Text'] = dataset['Text'].apply(remove_tags)

dataset.head()

,Category,Text,CategoryId
0,WELLNESS,143 Miles in 35 Days: Lessons LearnedResting i...,8
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?Th...,8
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,8
3,WELLNESS,"Oh, What a Difference She MadeIf you want to b...",8
4,WELLNESS,"Green SuperfoodsFirst, the bad news: Soda brea...",8


In [12]:
def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
dataset['Text'] = dataset['Text'].apply(special_char)

dataset.head()

,Category,Text,CategoryId
0,WELLNESS,143 Miles in 35 Days Lessons LearnedResting i...,8
1,WELLNESS,Talking to Yourself Crazy or Crazy Helpful Th...,8
2,WELLNESS,Crenezumab Trial Will Gauge Whether Alzheimer...,8
3,WELLNESS,Oh What a Difference She MadeIf you want to b...,8
4,WELLNESS,Green SuperfoodsFirst the bad news Soda brea...,8


In [13]:
def convert_lower(text):
   return text.lower()
dataset['Text'] = dataset['Text'].apply(convert_lower)
dataset

,Category,Text,CategoryId
0,WELLNESS,143 miles in 35 days lessons learnedresting i...,8
1,WELLNESS,talking to yourself crazy or crazy helpful th...,8
2,WELLNESS,crenezumab trial will gauge whether alzheimer...,8
3,WELLNESS,oh what a difference she madeif you want to b...,8
4,WELLNESS,green superfoodsfirst the bad news soda brea...,8
...,...,...,...
49995,SPORTS,this baseball team learned there s a wrong way...,5
49996,SPORTS,some young spurs fan dabbed 38 times in a sing...,5
49997,SPORTS,rasheed wallace ejected from knicks suns game ...,5
49998,SPORTS,why jake plummer and other nfl players are pus...,5


In [14]:
def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return [x for x in words if x not in stop_words]

dataset['Text'] = dataset['Text'].apply(remove_stopwords)
dataset

,Category,Text,CategoryId
0,WELLNESS,"[143, miles, 35, days, lessons, learnedresting...",8
1,WELLNESS,"[talking, crazy, crazy, helpful, think, talkin...",8
2,WELLNESS,"[crenezumab, trial, gauge, whether, alzheimer,...",8
3,WELLNESS,"[oh, difference, madeif, want, busy, keep, try...",8
4,WELLNESS,"[green, superfoodsfirst, bad, news, soda, brea...",8
...,...,...,...
49995,SPORTS,"[baseball, team, learned, wrong, way, celebrat...",5
49996,SPORTS,"[young, spurs, fan, dabbed, 38, times, single,...",5
49997,SPORTS,"[rasheed, wallace, ejected, knicks, suns, game...",5
49998,SPORTS,"[jake, plummer, nfl, players, pushing, researc...",5


In [15]:
def lemmatize_word(text):
  wordnet = WordNetLemmatizer()
  return " ".join([wordnet.lemmatize(word) for word in text])

dataset['Text'] = dataset['Text'].apply(lemmatize_word)
dataset



,Category,Text,CategoryId
0,WELLNESS,143 mile 35 day lesson learnedresting part tra...,8
1,WELLNESS,talking crazy crazy helpful think talking tool...,8
2,WELLNESS,crenezumab trial gauge whether alzheimer drug ...,8
3,WELLNESS,oh difference madeif want busy keep trying per...,8
4,WELLNESS,green superfoodsfirst bad news soda bread corn...,8
...,...,...,...
49995,SPORTS,baseball team learned wrong way celebrate japa...,5
49996,SPORTS,young spur fan dabbed 38 time single playoff g...,5
49997,SPORTS,rasheed wallace ejected knicks sun game yellin...,5
49998,SPORTS,jake plummer nfl player pushing research canna...,5


In [16]:
df = pd.DataFrame(dataset, columns= ['Category', 'Text', 'CategoryId'])

df.to_csv('/content/drive/MyDrive/Python-Dataset/model/file1.csv')
from joblib import dump, load
# dump(dataset, '/content/drive/MyDrive/Python-Dataset/model/dataset.csv') 
# oneVsRest = load('oneVsRest.joblib')

In [17]:
x = dataset['Text']
y = dataset['CategoryId']

print("x=\n", x)
print("\ny=\n", y)

x=
 0        143 mile 35 day lesson learnedresting part tra...
1        talking crazy crazy helpful think talking tool...
2        crenezumab trial gauge whether alzheimer drug ...
3        oh difference madeif want busy keep trying per...
4        green superfoodsfirst bad news soda bread corn...
                               ...                        
49995    baseball team learned wrong way celebrate japa...
49996    young spur fan dabbed 38 time single playoff g...
49997    rasheed wallace ejected knicks sun game yellin...
49998    jake plummer nfl player pushing research canna...
49999    simone bile next anyone first simone bile gymn...
Name: Text, Length: 50000, dtype: object

y=
 0        8
1        8
2        8
3        8
4        8
        ..
49995    5
49996    5
49997    5
49998    5
49999    5
Name: CategoryId, Length: 50000, dtype: int64


In [18]:
# from sklearn.feature_extraction.text import CountVectorizer
# # list of text documents
# text = ["The quick brown fox jumped over the lazy puppy."]
# # create the transform
# vectorizer = CountVectorizer()
# # tokenize and build vocab
# vectorizer.fit(text)
# # summarize
# print(vectorizer.vocabulary_)
# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(vector.shape)
# print(type(vector))
# print(vector.toarray())
# print(vectorizer.vocabulary_)

# text2 = ["the puppy"]
# vector = vectorizer.transform(text2)
# print(vector.toarray())

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

x = np.array(dataset.iloc[:,0].values)
y = np.array(dataset.CategoryId.values)

print("x=\n", x)
print("\ny=\n", y)

cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(dataset.Text).toarray()

print("x=\n", x)
print("\ny=\n", y)
print("\nx.shape = ",x.shape)
print("y.shape = ",y.shape)

x=
 ['WELLNESS' 'WELLNESS' 'WELLNESS' ... 'SPORTS' 'SPORTS' 'SPORTS']

y=
 [8 8 8 ... 5 5 5]
x=
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

y=
 [8 8 8 ... 5 5 5]

x.shape =  (50000, 5000)
y.shape =  (50000,)


In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, shuffle=True)

print("x-train = ",x_train)
print("x-test = ",x_test)

x-train =  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
x-test =  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### **Multinomial NaiveBayes**

In [21]:
# mdl = MultinomialNB(alpha=1.0,fit_prior=True)

# oneVsRest = OneVsRestClassifier(mdl)
# oneVsRest.fit(x_train, y_train)

# y_pred = oneVsRest.predict(x_test)

# y_pred

In [22]:
# accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

# print("Accuracy = ", accuracy)

# pricison, recall, f1score, support = score(y_test, y_pred, average='micro')

# print("Pricison = ", pricison)
# print("Recall = ", recall)
# print("F1Score = ", f1score)

### **Models**

In [23]:
perform_list = [ ]

# oneVsRest = OneVsRestClassifier(GaussianNB())
# oneVsRest.fit(x_train, y_train)

oneVsRest = None

def run_model(model_name, est_c, est_pnlty):
  mdl=''
  global oneVsRest;

  if model_name == 'Random Forest':

    mdl = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)

  elif model_name == 'Multinomial Naive Bayes':

    mdl = MultinomialNB(alpha=1.0,fit_prior=True)

  elif model_name == 'Decision Tree Classifier':

    mdl = DecisionTreeClassifier(criterion='entropy')

  elif model_name == 'Gaussian Naive Bayes':

    mdl = GaussianNB()

  oneVsRest = OneVsRestClassifier(mdl)

  oneVsRest.fit(x_train, y_train)

  y_pred = oneVsRest.predict(x_test)

# Performance metrics

  accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

# Get precision, recall, f1 scores

  precision, recall, f1score, support = score(y_test, y_pred, average='micro')

  print(f'Test Accuracy Score of Basic {model_name}: % {accuracy}')

  print(f'Precision : {precision}')

  print(f'Recall : {recall}')

  print(f'F1-score : {f1score}')

  # Add performance parameters to list

  perform_list.append(dict([

  ('Model', model_name),

  ('Test Accuracy', round(accuracy, 2)),

  ('Precision', round(precision, 2)),

  ('Recall', round(recall, 2)),

  ('F1', round(f1score, 2))

]))

In [24]:
# run_model('Random Forest', est_c=None, est_pnlty=None)
run_model('Multinomial Naive Bayes', est_c=None, est_pnlty=None)
# run_model('Decision Tree Classifier', est_c=None, est_pnlty=None)
# run_model('Gaussian Naive Bayes', est_c=None, est_pnlty=None)

model_performance = pd.DataFrame(data=perform_list)
model_performance = model_performance[['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1']]
model_performance

Test Accuracy Score of Basic Multinomial Naive Bayes: % 78.12
Precision : 0.7812
Recall : 0.7812
F1-score : 0.7812000000000001


,Model,Test Accuracy,Precision,Recall,F1
0,Multinomial Naive Bayes,78.12,0.78,0.78,0.78


In [25]:
# mdl = MultinomialNB(alpha=1.0,fit_prior=True)

# oneVsRest = OneVsRestClassifier(mdl)
# oneVsRest.fit(x_train, y_train)

# y_pred = oneVsRest.predict(x_test)
# y_pred

# classifier = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0).fit(x_train, y_train)
# classifier
# y_pred = classifier.predict(x_test)

In [40]:
y_pred1 = cv.transform(["actress want to make beauty parler such that so they can travel around the world go to beauty shop."])
print(y_pred1)
print(y_pred1.toarray())
# print(len(y_pred1))
# yy = classifier.predict(y_pred1)

#store model in DRIVE
from joblib import dump, load
# dump(oneVsRest, '/content/drive/MyDrive/Python-Dataset/model/oneVsRest.joblib') 

# Use when oneVsRest.joblib file uploded locally
# oneVsRest = load('oneVsRest.joblib') 

yy = oneVsRest.predict(y_pred1)

result = ""

if yy == [0]:
  result = "WellNess News"
elif yy == [1]:
  result = "POLITICS News"
elif yy == [2]:
  result = "ENTERTAINMENT News"
elif yy == [3]:
  result = "TRAVEL News"
elif yy == [4]:
  result = "STYLE & BEAUTY News"
elif yy == [5]:
  result = "PARENTING News"
elif yy == [6]:
  result = "FOOD & DRINK News"
elif yy == [7]:
  result = "WORLD News"
elif yy == [8]:
  result = "BUSINESS News"
elif yy == [9]:
  result = "SPORTS News"
print(result)

  (0, 120)	1
  (0, 330)	1
  (0, 469)	2
  (0, 1979)	1
  (0, 2758)	1
  (0, 4069)	1
  (0, 4635)	1
  (0, 4844)	1
  (0, 4950)	1
[[0 0 0 ... 0 0 0]]
FOOD & DRINK News


In [ ]:
# !pip install fastapi
# !pip install colabcode

In [32]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import pickle

class Data(BaseModel):
    sentence: str

app = FastAPI(debug=True)

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get('/')
def hello():
  print("---------------------------")
  print("Root Route called ---------------------")
  print("---------------------------")
  return "Hi There"

@app.post('/')
def get_music_category(data: Data):
  data = data.dict()
  y_pred1 = cv.transform([data['sentence']])
  # yy = classifier.predict(y_pred1)
  yy = oneVsRest.predict(y_pred1)

  if yy == [0]:
    result = "WellNess News"
  elif yy == [1]:
    result = "POLITICS News"
  elif yy == [2]:
    result = "ENTERTAINMENT News"
  elif yy == [3]:
    result = "TRAVEL News"
  elif yy == [4]:
    result = "STYLE & BEAUTY News"
  elif yy == [5]:
    result = "PARENTING News"
  elif yy == [6]:
    result = "FOOD & DRINK News"
  elif yy == [7]:
    result = "WORLD News"
  elif yy == [8]:
    result = "BUSINESS News"
  elif yy == [9]:
    result = "SPORTS News"
  print(result) 
  return {'prediction': result}

from colabcode import ColabCode
server = ColabCode(port=20000, code=False)
server.run_app(app=app)

INFO:     Started server process [57]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:20000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://f70b-35-194-21-240.ngrok.io" -> "http://localhost:20000"
---------------------------
Root Route called ---------------------
---------------------------
INFO:     49.44.76.134:0 - "GET / HTTP/1.1" 200 OK
---------------------------
Root Route called ---------------------
---------------------------
INFO:     2409:4041:e93:e6c7:605d:cd1d:6111:18b3:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4041:e93:e6c7:605d:cd1d:6111:18b3:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [57]
